In [1]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
# Carregando Dados
file_path = 'C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

In [5]:

# coluna de data em datetime
data['Data Referência'] = pd.to_datetime(data['Data Referência'])

In [6]:
# Criar a variável-alvo (IPCA Próximo Mês)
data['IPCA Próximo Mês'] = data['Valor IPCA'].shift(-1)
data = data.dropna()  # Remover valores ausentes


In [7]:

X = data.drop(columns=['Data Referência', 'IPCA Próximo Mês'])
y = data['IPCA Próximo Mês']

In [8]:
# Definir os modelos - ADICIONAR REGRESSÃO LINEAR REGULARIZADA
models = {
    "Regressão Linear": LinearRegression(),
    "SVR": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Random Forest": RandomForestRegressor()
}

In [9]:
# Definir grids de parâmetros para cada modelo
param_grids = {
    "SVR": {
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'epsilon': [0.001, 0.01, 0.1, 0.5, 1]
    },
    "Decision Tree": {
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    "Gradient Boosting": {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.001, 0.01, 0.1, 0.5],
        'max_depth': [3, 5, 10],
        'subsample': [0.8, 1.0]
    },
    "Random Forest": {
        'n_estimators': [100, 200, 500],
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
}

In [10]:
# Configuração do TimeSeriesSplit
n_splits = 5  # Número de divisões (folds)
tss = TimeSeriesSplit(n_splits=n_splits)


In [11]:
# Loop para treinar e avaliar cada modelo com GridSearchCV - C0LOCAR REFIT FALSO, OLHAR O ATRIBUTO .SCORE E BESTSCORE
for model_name, model in models.items():
    print(f"Treinando e avaliando o modelo: {model_name}")
    
    # Configurar o GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grids.get(model_name, {}),
        scoring='neg_mean_squared_error',  # Avaliar pelo erro quadrático médio negativo
        cv=tss,                            # TimeSeriesSplit para validação cruzada temporal
        verbose=1,                         # Exibe progresso
        n_jobs=-1                          # Usa todos os núcleos disponíveis para acelerar o processo
    )
    
    # Ajustar no conjunto de dados
    grid_search.fit(X, y)
    
    # Resultados
    print(f"Melhores Parâmetros para o modelo {model_name}: {grid_search.best_params_}")
    print(f"Melhor Pontuação (MSE Negativo) para o modelo {model_name}: {grid_search.best_score_:.4f}")
    print("=" * 50)
    
    # Avaliar o modelo final com os melhores parâmetros
    best_model = grid_search.best_estimator_
    
    # Calcular MSE e R² finais com o modelo otimizado
    y_pred = best_model.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    print(f"Desempenho final para o modelo {model_name}:")
    print(f"  MSE: {mse:.4f}")
    print(f"  R²: {r2:.4f}")
    print("=" * 50)

Treinando e avaliando o modelo: Regressão Linear
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Melhores Parâmetros para o modelo Regressão Linear: {}
Melhor Pontuação (MSE Negativo) para o modelo Regressão Linear: -0.1191
Desempenho final para o modelo Regressão Linear:
  MSE: 0.0005
  R²: 0.9943
Treinando e avaliando o modelo: SVR
Fitting 5 folds for each of 120 candidates, totalling 600 fits
Melhores Parâmetros para o modelo SVR: {'C': 0.1, 'epsilon': 0.001, 'kernel': 'linear'}
Melhor Pontuação (MSE Negativo) para o modelo SVR: -0.0508
Desempenho final para o modelo SVR:
  MSE: 0.0021
  R²: 0.9739
Treinando e avaliando o modelo: Decision Tree
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Melhores Parâmetros para o modelo Decision Tree: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
Melhor Pontuação (MSE Negativo) para o modelo Decision Tree: -0.0825
Desempenho final para o modelo Decision Tree:
  MSE: 0.0004
  R²: 0.9948
Treinando e avali